In [9]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
data_alert_bogor = pl.read_csv('../data/raw/aggregate_alerts_Kota Bogor.csv').to_pandas()
data_med_irr_bogor = pl.read_csv('../data/raw/aggregate_median_irregularities_Kota Bogor.csv').to_pandas()
data_med_jam_bogor = pl.read_csv('../data/raw/aggregate_median_jams_Kota Bogor.csv').to_pandas()

In [11]:
data_alert_bogor.sample(7)

,id,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,type,avg_location,total_records,date
8871,6190016,2022-08-30 17:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Tajur,JAM,"[106.81825000000002, -6.622948]",7,2022-08-30
7084,6137868,2022-08-19 20:00:00.000,32.71,KOTA BOGOR,Semplak Raya,WEATHERHAZARD,"[106.76053421367521, -6.548724307692318]",117,2022-08-19
138,5930314,2022-07-06 18:00:00.000,32.71,KOTA BOGOR,N9 Ir Haji Juanda,JAM,"[106.79421864705877, -6.595595176470586]",34,2022-07-06
2271,6007772,2022-07-22 07:00:00.000,32.71,KOTA BOGOR,TB M Falak,JAM,"[106.77745899999994, -6.5836209999999955]",30,2022-07-22
1286,5965867,2022-07-14 21:00:00.000,32.71,KOTA BOGOR,Tol Jagorawi,WEATHERHAZARD,"[106.83356199999996, -6.617885]",26,2022-07-14
6205,6111927,2022-08-13 20:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Pajajaran,JAM,"[106.80527399999995, -6.587921999999999]",31,2022-08-13
4302,6065014,2022-08-03 13:00:00.000,32.71,KOTA BOGOR,Arah Sentul Selatan/Tol Lingkar Bogor/Jakarta,WEATHERHAZARD,"[106.83004200000002, -6.595517]",8,2022-08-03


In [12]:
data_alert_bogor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9762 entries, 0 to 9761
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         9762 non-null   int64  
 1   time                       9762 non-null   object 
 2   kemendagri_kabupaten_kode  9762 non-null   float64
 3   kemendagri_kabupaten_nama  9762 non-null   object 
 4   street                     8827 non-null   object 
 5   type                       9762 non-null   object 
 6   avg_location               9762 non-null   object 
 7   total_records              9762 non-null   int64  
 8   date                       9762 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 686.5+ KB


In [ ]:
data_alert_bogor['time'] = data_alert_bogor['time']

In [21]:
categorical_col_alert_bogor = [
    column
    for column in data_alert_bogor.columns
    if len(data_alert_bogor[column].unique()) <= 100
]

In [27]:
for col in categorical_col_alert_bogor:
    print(f'Unique values in {col}: \n{data_alert_bogor[col].unique()}\n')

Unique values in kemendagri_kabupaten_kode: 
[32.71]

Unique values in kemendagri_kabupaten_nama: 
['KOTA BOGOR']

Unique values in type: 
['ROAD_CLOSED' 'WEATHERHAZARD' 'JAM' 'ACCIDENT']

Unique values in date: 
['2022-07-06' '2022-07-07' '2022-07-08' '2022-07-09' '2022-07-10'
 '2022-07-11' '2022-07-12' '2022-07-13' '2022-07-14' '2022-07-15'
 '2022-07-16' '2022-07-17' '2022-07-18' '2022-07-19' '2022-07-20'
 '2022-07-21' '2022-07-22' '2022-07-23' '2022-07-24' '2022-07-25'
 '2022-07-26' '2022-07-27' '2022-07-28' '2022-07-29' '2022-07-30'
 '2022-07-31' '2022-08-01' '2022-08-02' '2022-08-03' '2022-08-04'
 '2022-08-05' '2022-08-06' '2022-08-07' '2022-08-08' '2022-08-09'
 '2022-08-10' '2022-08-11' '2022-08-12' '2022-08-13' '2022-08-14'
 '2022-08-15' '2022-08-16' '2022-08-17' '2022-08-18' '2022-08-19'
 '2022-08-20' '2022-08-21' '2022-08-22' '2022-08-23' '2022-08-24'
 '2022-08-25' '2022-08-26' '2022-08-27' '2022-08-28' '2022-08-29'
 '2022-08-30' '2022-08-31' '2022-09-01' '2022-09-02' '2022-09

- Terdapat missing values pada kolom 'street', sehingga perlu dilakukan handling
- Kolom 'date' tidak terlalu penting karena sudah ada kolom time
- Kolom 'kemendagri_kabupaten_kode' dan 'kemendagri_kabupaten_nama' hanya memiliki 1 unique value jadi akan kita drop nanti
- Data tersedia dalam rentang 2 bulan dari 6 Juli 2022 hingga 6 September 2022
  - Terdapat 4 unique value di kolom 'type yaitu:
  - 'ROAD_CLOSED': Jalan Ditutup,
  - 'WEATHERHAZARD': Cuaca Buruk,
  - 'JAM': Macet,
  - 'ACCIDENT': Kecelakaan

In [13]:
data_med_irr_bogor.sample(7)

,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,jam_level,median_length,median_delay_seconds,median_regular_speed,total_records,cause_type,median_seconds,median_speed,date,median_jam_level,id,geometry
1957,2022-08-06 18:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Dramaga,4,998.0,563.0,16.830000,2,None,652.0,5.500,2022-08-06,4.0,1544243,"LINESTRING (106.72843 -6.562546, 106.728643 -6..."
3233,2022-08-27 15:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Ciawi,4,744.0,621.0,11.600000,2,None,686.0,3.890,2022-08-27,4.0,1569760,"LINESTRING (106.847424 -6.65615, 106.848115 -6..."
2504,2022-08-13 17:00:00.000,32.71,KOTA BOGOR,Dr Sumeru,4,1161.5,542.5,22.279999,4,None,664.0,6.230,2022-08-13,4.0,1554049,"MULTILINESTRING ((106.777404 -6.581281, 106.77..."
44,2022-07-06 18:00:00.000,32.71,KOTA BOGOR,Pandawa Raya,4,535.0,444.0,21.150000,16,None,508.0,3.785,2022-07-06,4.0,1492836,"MULTILINESTRING ((106.815074 -6.570535, 106.81..."
1302,2022-07-29 18:00:00.000,32.71,KOTA BOGOR,N8 Jalan Raya Bogor,4,1842.0,653.0,18.455000,14,None,847.0,7.685,2022-07-29,4.0,1530808,"MULTILINESTRING ((106.825415 -6.545147, 106.82..."
758,2022-07-17 18:00:00.000,32.71,KOTA BOGOR,Pulo Empang,4,590.0,460.0,17.760000,11,None,544.0,3.890,2022-07-17,4.0,1515217,"MULTILINESTRING ((106.793492 -6.607427, 106.79..."
1727,2022-08-02 18:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Pajajaran,3,1056.0,194.0,19.830000,1,None,278.0,13.650,2022-08-02,3.0,1538197,"LINESTRING (106.807039 -6.579006, 106.807091 -..."


In [14]:
data_med_irr_bogor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4051 entries, 0 to 4050
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   time                       4051 non-null   object 
 1   kemendagri_kabupaten_kode  4051 non-null   float64
 2   kemendagri_kabupaten_nama  4051 non-null   object 
 3   street                     4051 non-null   object 
 4   jam_level                  4051 non-null   int64  
 5   median_length              4051 non-null   float64
 6   median_delay_seconds       4051 non-null   float64
 7   median_regular_speed       4051 non-null   float64
 8   total_records              4051 non-null   int64  
 9   cause_type                 0 non-null      object 
 10  median_seconds             4051 non-null   float64
 11  median_speed               4051 non-null   float64
 12  date                       4051 non-null   object 
 13  median_jam_level           4051 non-null   float

In [15]:
data_med_jam_bogor.sample(7)

,,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,id,date,median_level,geometry
29624,29624,2022-07-24 12:00:00.000,32.71,KOTA BOGOR,Ence Sumartadireja,2,712.0,75.5,14.770000,8,34645232,2022-07-24,2.0,"MULTILINESTRING ((106.807857 -6.633475, 106.80..."
74959,74959,2022-08-20 10:00:00.000,32.71,KOTA BOGOR,N9 Jalak Harupat,3,1026.0,372.0,7.840000,39,36309499,2022-08-20,3.0,"MULTILINESTRING ((106.79706 -6.593115, 106.797..."
50242,50242,2022-08-05 11:00:00.000,32.71,KOTA BOGOR,KH Soleh Iskandar (Jalur Lambat),3,747.0,120.0,15.120000,5,35391119,2022-08-05,3.0,"MULTILINESTRING ((106.787564 -6.560208, 106.78..."
20601,20601,2022-07-18 17:00:00.000,32.71,KOTA BOGOR,Cilebut Raya,1,908.0,65.0,19.130000,17,34263956,2022-07-18,1.0,"MULTILINESTRING ((106.798965 -6.548614, 106.79..."
45365,45365,2022-08-02 11:00:00.000,32.71,KOTA BOGOR,Dr Sumeru,2,1491.0,86.0,20.300000,21,35213540,2022-08-02,2.0,"MULTILINESTRING ((106.775836 -6.579968, 106.77..."
47980,47980,2022-08-03 20:00:00.000,32.71,KOTA BOGOR,Kolonel Ahmad Syam,2,958.0,85.5,19.189999,8,35270833,2022-08-03,2.0,"MULTILINESTRING ((106.818162 -6.600064, 106.81..."
62495,62495,2022-08-12 13:00:00.000,32.71,KOTA BOGOR,RE Martadinata,4,217.0,116.0,5.560000,3,35831583,2022-08-12,4.0,"MULTILINESTRING ((106.790175 -6.58282, 106.790..."


In [16]:
data_med_jam_bogor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102322 entries, 0 to 102321
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0                              102322 non-null  int64  
 1   time                       102322 non-null  object 
 2   kemendagri_kabupaten_kode  102322 non-null  float64
 3   kemendagri_kabupaten_nama  102322 non-null  object 
 4   street                     100529 non-null  object 
 5   level                      102322 non-null  int64  
 6   median_length              102322 non-null  float64
 7   median_delay               102322 non-null  float64
 8   median_speed_kmh           102322 non-null  float64
 9   total_records              102322 non-null  int64  
 10  id                         102322 non-null  int64  
 11  date                       102322 non-null  object 
 12  median_level               102322 non-null  float64
 13  geometry                   10